In [ ]:
# @title 1.a Setup environment
!pip install -q --upgrade --quiet torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!apt -y install -qq aria2

# Clone the OFFICIAL ComfyUI repository for maximum stability
!git clone https://github.com/comfyanonymous/ComfyUI.git /content/ComfyUI
!pip install -q -r /content/ComfyUI/requirements.txt

# GGUF custom nodes from calcuis
!git clone https://github.com/calcuis/gguf.git /content/ComfyUI/custom_nodes/gguf
#no requirements.txt

print("------------------------")
print("✅ Environment Setup Complete!")

In [ ]:
# @title 1.b Select Model

from enum import Enum

class Architecture(Enum):
    CHECKPOINT = "checkpoint"
    MODULAR = "modular"

def download_model(model_entry, component_type, dest_comfyui_dir):

    component = model_entry.get(component_type)

    if component.get("custom_url", False) == True:
      download_url =component.get('url')
    else:
      download_url = component.get('url') + component.get('name')

    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M "{download_url}" -d "/content/ComfyUI/models/{dest_comfyui_dir}" -o "{component.get('name')}"

selected_model = "base"

models = {
  "AbsReal181":{
    "architecture": Architecture.CHECKPOINT,
    "base_size_px": 512,
    "type": "SD15",
    "checkpoint":{
      "name": "AbsoluteReality_1.8.1_pruned.safetensors",
      "url": "https://huggingface.co/Lykon/AbsoluteReality/resolve/main/"
      }
    },
  "Delib6":{
    "architecture": Architecture.CHECKPOINT,
    "base_size_px": 512,
    "type": "SD15",
    "checkpoint":{
      "name": "Deliberate_v6.safetensors",
      "url": "https://huggingface.co/XpucT/Deliberate/resolve/main/"
      }
    },
  "DreamS8":{
    "architecture": Architecture.CHECKPOINT,
    "base_size_px": 512,
    "type": "SD15",
    "checkpoint":{
      "name": "DreamShaper_8_pruned.safetensors",
      "url": "https://huggingface.co/Lykon/DreamShaper/resolve/main/"
      }
    },
  "epicR_NS":{
    "architecture": Architecture.CHECKPOINT,
    "base_size_px": 512,
    "type": "SD15",
    "checkpoint":{
      "name": "epicrealism_naturalSin.safetensors",
      "url": "https://huggingface.co/TheImposterImposters/epiCRealism-NaturalSin/resolve/main/"
      }
    },
  "RV60B1":{
    "architecture": Architecture.CHECKPOINT,
    "base_size_px": 512,
    "use_custom_vae": True,
    "type": "SD15",
    "checkpoint":{
      "name": "Realistic_Vision_V6.0_NV_B1_fp16.safetensors",
      "url": "https://huggingface.co/SG161222/Realistic_Vision_V6.0_B1_noVAE/resolve/main/"
      },
    "vae": {
      "name": "vae-ft-mse-840000-ema-pruned.safetensors",
      "url": "https://huggingface.co/stabilityai/sd-vae-ft-mse-original/resolve/main/"
      },
    },
  "ICBINP":{
    "architecture": Architecture.CHECKPOINT,
    "base_size_px": 512,
    "type": "SD15",
    "checkpoint":{
      "custom_url": True,
      "name": "icbinpICantBelieveIts_mid2024.safetensors",
      "url": "https://civitai.com/api/download/models/667760?type=Model&format=SafeTensor&size=pruned&fp=fp16"
      }
    },
  "HUNYDIT":{
    "architecture": Architecture.CHECKPOINT,
    "base_size_px": 1024,
    "type": "HUNYUAN_DIT",
    "checkpoint":{
      "custom_url": False,
      "name": "hunyuan_dit_1.2.safetensors",
      "url": "https://huggingface.co/comfyanonymous/hunyuan_dit_comfyui/resolve/main/"
      }
    },
  "LUMINA2":{
    "architecture": Architecture.MODULAR,
    "base_size_px": 1024,
    "type": "LUMINA2",
    "unet":{
      "name": "lumina2-q8_0.gguf",
      "url": "https://huggingface.co/calcuis/lumina-gguf/resolve/main/",
      "is_gguf": True
      },
    "vae": {
      "name": "diffusion_pytorch_model.safetensors",
      "url": "https://huggingface.co/Alpha-VLLM/Lumina-Image-2.0/resolve/main/vae/"
      },
    "text_encoder":{
      "name": "gemma_2_2b_fp16.safetensors",
      "url": "https://huggingface.co/calcuis/lumina-gguf/resolve/main/",
      "type": "LUMINA2",
      "is_gguf": False
      },
    },
  "base":{
    "architecture": Architecture.CHECKPOINT,
    "base_size_px": 512,
    "type": "SD15",
    "checkpoint":{
      "name": "v1-5-pruned-emaonly.safetensors",
      "url": "https://huggingface.co/stable-diffusion-v1-5/stable-diffusion-v1-5/resolve/main/"
      }
    },
  }

model_entry = models.get(selected_model)

if model_entry:
  if model_entry.get("architecture", "checkpoint") == Architecture.CHECKPOINT:

    download_model(model_entry, "checkpoint", "checkpoints")

  elif model_entry.get("architecture", "checkpoint") == Architecture.MODULAR:

    download_model(model_entry, "unet", "unet")

    download_model(model_entry, "text_encoder", "text_encoders")

  else:
    raise ValueError(f"No valid architecture selected.")

  if (model_entry.get("use_custom_vae", False) == True
    or model_entry.get("architecture", "checkpoint") == Architecture.MODULAR):

    download_model(model_entry, "vae", "vae")


  print("------------------------")
  print("✅ Model Download Complete!")
else:
  print("------------------------")
  print("❌ No valid model selected.")

In [ ]:
# @title 1.c LoRAs, Embeddings and Postprocessing
from google.colab import drive
import os

drive.mount('/content/drive')
!mkdir -p /content/ComfyUI/models/loras

lora_file_root = "/content/drive/MyDrive/AI/LoRAs/SD1.5"

loras_config = [
  {
    "filename": "sample_lora_1.safetensors",
    "strength": 1.0,
    "type": "SD15",
    "enabled": False
  },
  {
    "filename": "sample_lora_2.safetensors",
    "strength": 0.85,
    "type": "SD15",
    "enabled": True
  },
]

for lora_config in loras_config:
  if (lora_config.get("enabled", False) == True):
    lora_filename = lora_config["filename"]
    print(f"Copying {lora_file_root}/{lora_filename}")
    !cp "{lora_file_root}/{lora_filename}" "/content/ComfyUI/models/loras/"

# embeddings
if model_entry['type'] == "SD15":
  src_embeddings_folder = "/content/drive/MyDrive/AI/Embeddings/SD1.5/"
  with os.scandir(src_embeddings_folder) as entries:
    for entry in entries:
      if entry.is_file() and entry.name.endswith(".safetensors"):
        print(f"Copying {entry.path}")
        !cp "{entry.path}" "/content/ComfyUI/models/embeddings/"

# upscaling

upscale_model_name = "RealESRGAN_x4plus.safetensors"
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M "https://huggingface.co/Comfy-Org/Real-ESRGAN_repackaged/resolve/main/{upscale_model_name}" -d /content/ComfyUI/models/upscale_models -o {upscale_model_name}

print("✅ LoRAs, Embeddings and Postprocessing Setup Complete!")

In [ ]:
# @title 2. Logic

# Imports
import sys, os, gc, random, pdb
import torch
from datetime import datetime
from IPython.display import display, Image as IPImage

sys.path.insert(0, '/content/ComfyUI')

from nodes import (
    CheckpointLoaderSimple, VAELoader, KSampler, LatentUpscale,
    LoraLoader, EmptyLatentImage, VAEDecode, CLIPTextEncode,
    UNETLoader, CLIPLoader, SaveImage)
from comfy_extras.nodes_upscale_model import (
    UpscaleModelLoader, ImageUpscaleWithModel
)

from custom_nodes.gguf.pig import (
    LoaderGGUF, ClipLoaderGGUF, VaeGGUF
)

def generate_images(params):

  model_entry = models.get(selected_model)
  if not model_entry:
    raise ValueError(f"Model not found: The key '{selected_model}' does not exist in the models dictionary.")

  with torch.inference_mode():

    if (model_entry.get("architecture", "checkpoint") == Architecture.CHECKPOINT):

      model, text_encoder, vae = CheckpointLoaderSimple().load_checkpoint(
                                      model_entry["checkpoint"]["name"])

    elif (model_entry.get("architecture", "checkpoint") == Architecture.MODULAR):

      if (model_entry.get("unet").get("is_gguf")):

        model = LoaderGGUF().load_model(model_entry.get("unet").get("name"))[0]
      else:
        #don't forget to add wight_dtype to unet when needed in the dictionary
        #possible values "fp8_e4m3fn", "fp8_e4m3fn_fast", "fp8_e5m2"
        model = UNETLoader().load_unet(model_entry.get("unet").get("name"),
                                       model_entry.get("unet").get("weight_dtype"))[0]

      if (model_entry.get("text_encoder").get("is_gguf")):

        text_encoder = ClipLoaderGGUF().load_clip(model_entry.get("text_encoder").get("name"),
                      type=model_entry.get("text_encoder").get("type", "stable_diffusion"))[0]
      else:
        text_encoder = CLIPLoader().load_clip(model_entry.get("text_encoder").get("name"),
                       type=model_entry.get("text_encoder").get("type", "stable_diffusion"))[0]
    else:
      raise ValueError(f"No valid architecture selected.")

    clip_skip = int(params.get("clip_skip")) if params.get("clip_skip") else 1
    if clip_skip and clip_skip > 1:
      text_encoder.clip_layer(-clip_skip)

    if (model_entry.get("use_custom_vae", False) == True
        or (model_entry.get("architecture", "checkpoint") == Architecture.MODULAR)):
      print(f"Using custom VAE: {model_entry['vae']['name']}")
      vae = VAELoader().load_vae(model_entry['vae']['name'])[0]

    for lora_config in loras_config:
      lora_name = lora_config.get("filename")
      strength = float(lora_config.get("strength", 1.0))

      if (lora_config.get("enabled", False) == True and
        strength > 0.0 and
        model_entry.get("type", "SD15") == lora_config.get("type", "SD15")):

        patched_model, patched_clip = LoraLoader().load_lora(
            model = model,
            clip = text_encoder,
            lora_name = lora_name,
            strength_model = strength,
            strength_clip = strength
        )

        if strength > 0.0:
          #load_lora clones model and clip when strenght > 0.0
          del model, text_encoder
        model = patched_model
        text_encoder = patched_clip

    positive_cond = CLIPTextEncode().encode(text_encoder, params['prompt'])[0]
    negative_cond = CLIPTextEncode().encode(text_encoder, params['neg_prompt'])[0]

    base_size = int(model_entry.get('base_size_px', 512))

    final_width = int(params.get("width", base_size))
    final_height = int(params.get("height", base_size))

    final_steps = int(params.get("steps", 25))

    final_sampler  = params.get("sampler_name", "dpmpp_sde")
    final_scheduler = params.get("scheduler", "karras")

    if params.get("hires_fix", False) == True:

      first_width = (base_size if final_width < final_height
        else final_width * base_size / final_height)
      first_height = (base_size if final_width > final_height
        else final_height * base_size / final_width)

      first_width = round(first_width/8.0) * 8
      first_height = round(first_height/8.0) * 8

      first_steps = round(final_steps * 0.4)
      final_steps = final_steps - first_steps

      first_sampler = "dpmpp_sde_gpu"
      first_scheduler = "normal"

    else:
      first_width = final_width
      first_height = final_height
      first_steps = final_steps
      first_sampler = final_sampler
      first_scheduler = final_scheduler

    empty_latent_img = EmptyLatentImage().generate(
        first_width, first_height, params['batch_size'])[0]

    print(f"First KSampler pass with empty latent at W:{first_width}xH:{first_height}")
    print(f"with steps:{first_steps}")

    sampled_latent = KSampler().sample(
        model, params['seed'], first_steps, params['cfg'],
        first_sampler, first_scheduler,
        positive_cond, negative_cond,
        empty_latent_img, denoise=1.0)[0]

    if params.get("hires_fix", False) == True:

      upscaled_latent = LatentUpscale().upscale(sampled_latent, "nearest-exact",
                                       final_width, final_height, "disabled")[0]

      print(f"Second KSampler pass with upscaled latent at W:{final_width}xH:{final_height}")
      print(f"with steps:{final_steps}")

      final_latent = KSampler().sample(
        model, params['seed'], final_steps, params['cfg'],
        final_sampler, final_scheduler,
        positive_cond, negative_cond,
        upscaled_latent, denoise=0.5)[0]

      del upscaled_latent

    else:
      final_latent = sampled_latent


    del empty_latent_img
    del model, positive_cond, negative_cond

    if 'patched_model' in locals():
      del patched_model
    if 'patched_clip' in locals():
      del patched_clip

    images = VAEDecode().decode(vae, final_latent);

    del vae, sampled_latent, final_latent

    if params.get("upscale_with_model", False) == True:
      print(f"Upscaling with model: {upscale_model_name}")
      upscale_model = UpscaleModelLoader().load_model(upscale_model_name)[0]
      upscaled_images = ImageUpscaleWithModel().upscale(upscale_model, images[0])
      del images, upscale_model
      images = upscaled_images

    return images

def save_and_display_images(tensor_images, generation_params,
                            gdrive_output_path = None):

  if tensor_images is None or len(tensor_images) == 0:
    print("No image to save or display.")
    return

  with torch.inference_mode():

    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S.%f")[:-3]
    prefix = f"{timestamp}-"

    saved_images = []

    for t_image in tensor_images:
      cpu_image = t_image.detach().cpu()
      s_image = SaveImage().save_images(cpu_image, prefix, prompt=None,
                          extra_pnginfo=generation_params)
      saved_images.append(s_image['ui']['images'][0])

      if gdrive_output_path:
        source_path = os.path.join("/content/ComfyUI/output",
                                    saved_image['subfolder'],
                                    saved_image['filename'])
        !cp "{source_path}" "{gdrive_output_path}"

    print(f"Images saved")

  # Display the image in Colab
  for saved_image in saved_images:
    print("\n--- Generated Image ---")
    display(IPImage(filename= os.path.join("/content/ComfyUI/output",
                                           saved_image['subfolder'],
                                           saved_image['filename'])))

    print("\n--- Generation Details ---")
    print(f"Model: {selected_model}")
    for key, value in generation_params.items():
        print(f"{key.replace('_', ' ').capitalize()}: {value}")
    print("-------------------------")

def clean_memory():

  gc.collect()
  if torch.cuda.is_available():
      torch.cuda.empty_cache()
      torch.cuda.ipc_collect()
  for obj in list(globals().values()):
      if torch.is_tensor(obj) or (hasattr(obj, "data") and torch.is_tensor(obj.data)):
          del obj
  gc.collect()

print("✅ Logic Setup Complete!")

In [ ]:
# @title Pre-3. Saved Prompts
sp_select = 1

s_p = ""
s_n_p = ""

if sp_select == 1:
  s_p = ("DSLR photography of a black kitten playing with a ball of yarn")
  s_n_p = ("worst quality, low quality, painting, drawing, sketch, cartoon,"
           "anime, manga, render, CG, 3d, blurry, deformed, disfigured,"
           "mutated, bad anatomy, bad art, watermark, signature, label")

elif sp_select == 2:
  s_p = ("(embedding:fcDetailPortrait:1.0), (embedding:AS-YoungV2:1.2), "
    "realistic professional photographic portrait of a young Woman "
    "wearing a black Turtleneck, masterpiece, bokeh, DSLR photography")
  s_n_p = ("(embedding:BadDream:1.0), (embedding:UnrealisticDream:1.2), "
    "(embedding:FastNegativeV2:1.0)")

print("✅ Stored prompt selected!")

In [ ]:
# @title 3. Interface

prompt = "DSLR photography of a black kitten playing with a ball of yarn" # @param {"type":"string"}
if s_p:
  prompt = s_p

neg_prompt = "3D, cartoon, painting, bad quality, low quality" # @param {"type":"string"}
if s_n_p:
  neg_prompt = s_n_p

width = 512 # @param {"type":"slider", "min":"512", "max":"2048", "step": "8"}
height = 768 # @param {"type":"slider", "min":"512", "max":"2048", "step": "8"}
steps = 20 # @param {"type":"number"}
cfg = 7.5 # @param {"type":"number"}
sampler_name = "dpmpp_2m_sde_gpu" # @param ["euler", "euler_cfg_pp", "euler_ancestral", "euler_ancestral_cfg_pp", "heun", "heunpp2","dpm_2", "dpm_2_ancestral", "lms", "dpm_fast", "dpm_adaptive", "dpmpp_2s_ancestral", "dpmpp_2s_ancestral_cfg_pp", "dpmpp_sde", "dpmpp_sde_gpu", "dpmpp_2m", "dpmpp_2m_cfg_pp", "dpmpp_2m_sde", "dpmpp_2m_sde_gpu", "dpmpp_3m_sde", "dpmpp_3m_sde_gpu", "ddpm", "lcm", "ipndm", "ipndm_v", "deis", "res_multistep", "res_multistep_cfg_pp", "res_multistep_ancestral", "res_multistep_ancestral_cfg_pp", "gradient_estimation", "gradient_estimation_cfg_pp", "er_sde", "seeds_2", "seeds_3", "sa_solver", "sa_solver_pece"]
scheduler = "karras" # @param ["simple", "sgm_uniform", "karras", "exponential", "ddim_uniform", "beta", "normal", "linear_quadratic", "kl_optimal"]
hires_fix = False # @param{"type":"boolean"}
upscale_with_model = False # @param{"type":"boolean"}
batch_size = 1 # @param {"type":"number"}
seed = 0 # @param {"type":"number"}
clip_skip = 1 # @param {"type" : "number" }
save_to_gdrive = False # @param{"type":"boolean"}
gdrive_path = "/content/drive/MyDrive/AI/Images/SD1.5/" # @param{"type":"string"}

seed = seed if seed else random.randint(0, 2**32 - 1)
print(f"Seed : {seed}")

generation_params = {
    "prompt": prompt,
    "neg_prompt": neg_prompt,
    "width": width,
    "height": height,
    "steps": steps,
    "cfg": cfg,
    "sampler_name": sampler_name,
    "scheduler": scheduler,
    "hires_fix" : hires_fix,
    "upscale_with_model" : upscale_with_model,
    "batch_size": batch_size,
    "seed": seed,
    "clip_skip": clip_skip
}

#import warnings
#warnings.filterwarnings('ignore')

if save_to_gdrive:
  drive.mount('/content/drive')
  !mkdir -p "{gdrive_path}"

images = generate_images(generation_params)
save_and_display_images(images, generation_params,
                        gdrive_path if save_to_gdrive else None)

del images
clean_memory()

In [ ]:
# @title Clean Memory
clean_memory()